In [1]:
import pandas as pd
import os, sys 

https://medium.com/@sandyshah1990/langchain-agents-and-function-calling-using-llama-2-locally-29ce057e4789

### Libraries

In [2]:
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS

### Load  requirements

In [3]:
# Carregar o arquivo Excel
file_path = "Template - Requisito de Software v02.xlsx"
xls = pd.ExcelFile(file_path)

# Carregar abas específicas
sobre_documento = pd.read_excel(xls, sheet_name="Sobre o documento")
glossario = pd.read_excel(xls, sheet_name="1-Glossário")
requisitos = pd.read_excel(xls, sheet_name='2-Requisitos de Produto', skiprows= len(xls.sheet_names)+1)
lists = pd.read_excel(xls, sheet_name="3-Lists")
versoes = pd.read_excel(xls, sheet_name="4-Versões")
guia = pd.read_excel(xls, sheet_name="5-Guia")

requisitos= requisitos.dropna(subset=['Requisito','Categoria'])
# Exibir as primeiras linhas da aba de requisitos
#print(requisitos.head())

requisitos_texto = requisitos.to_string(index=False)

/opt/conda/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [4]:
requisitos.head()

,Req ID,Categoria,Descrição Alto Nível,Requisito,Comentários,Status,Histórico,Versão de modificação,Origem,Jira issue,Test Case Jira
0,PRS-00001,Requisitos Funcionais,RF - Modos de Operação,O dispositivo deve ser capaz de operar em cinc...,NaN,Novo,NaN,v_1,NaN,NaN,NaN
1,PRS-00002,Requisitos Funcionais,RF - Modos de Operação,O dispositivo deve permitir a ativação do modo...,configurado pelo Json,Novo,NaN,v_1,NaN,NaN,NaN
2,PRS-00003,Requisitos Funcionais,RF - Modos de Operação,O dispositivo deve permitir a ativação do modo...,configurado pelo Json,Novo,NaN,v_1,NaN,NaN,NaN
3,PRS-00004,Requisitos Funcionais,RF - Modos de Operação,O dispositivo deve permitir a ativação do modo...,configurado pelo Json,Novo,NaN,v_1,NaN,NaN,NaN
4,PRS-00005,Requisitos Funcionais,RF - Modos de Operação,O dispositivo deve permitir a ativação do modo...,configurado pelo Json,Novo,NaN,v_1,NaN,NaN,NaN


### LLM

In [5]:
import json
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import TransformChain, LLMChain
import json

# Função para converter resposta em JSON
def resp_to_json(resp):
    limpa_string = resp.replace('```json', '').replace('```', '').strip()
    try:
        dados = json.loads(limpa_string)
        return dados
    except json.JSONDecodeError as e:
        print("Erro ao converter para JSON:", e)
        return None

# Usando TransformChain para aplicar o resp_to_json
json_transform = TransformChain(
    input_variables=["text"],
    output_variables=["json"],
    transform=lambda inputs: {"json": resp_to_json(inputs["text"])}
)


In [10]:
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate

# Configurando o Ollama como LLM
llm = Ollama(model="llama3.2:1b")  # Substitua "llama2" pelo nome do modelo que você baixou

# Exemplo de prompt
prompt_template = PromptTemplate(
    input_variables=["requisitos"],
    template="""
    Você é um Analista de Requisitos. Revise os requisitos abaixo e avalie sua clareza e completude.
    Identifique quaisquer ambiguidades ou lacunas e forneça sugestões para melhorias.

    Requisitos:
    {requisitos}

    Relatório:
    
    """
    
)



In [12]:
# Função para gerar análise de clareza
def gerar_analise_clareza(requisitos):
    prompt = prompt_template.format(requisitos=requisitos)
    return llm(prompt)

# Testando a função
requisitos_produto =""  
for _, row in requisitos.iterrows():
    content = f"ID: {row['Req ID']} ; Requisito: {row['Requisito']}: \n"
    requisitos_produto+=content


# Requisitos de exemplo
#requisitos_produto = """
#1. O sistema deve permitir que os usuários façam login usando e-mail e senha.
#2. O sistema deve ser rápido.
#3. Os dados dos usuários devem ser armazenados de forma segura.
#4. O sistema deve funcionar em todos os dispositivos.
#"""


analise_clareza = gerar_analise_clareza(requisitos_produto)
print("Análise de Clareza:")
print(analise_clareza)

Análise de Clareza:
Segue um relatório detalhado sobre os requisitos fornecidos:

**Identificação de Ambiguidades e Lacunas**

1. **Requisito ID 00001**: É claro que o dispositivo deve ser capaz de operar em cinco modos de operação distintos (B, M, Doppler Colorido, Doppler Power, PW Doppler).
2. **Requisito ID 00002**: Só se implementa após a ativação do modo B quando houver uma solicitação através da API.
3. **Requisito ID 00003**: Implementado em todos os dispositivos.
4. **Requisito ID 00004**: Embora seja mencionado que não é necessário implementar o botão com sinalização para "DOWN", parece ser uma lacuna de exigência.

**Implementação dos requisitos**

1. **Requisito ID 00101**: O dispositivo deve possuir um botão de liga/desligue.
2. **Requisito ID 00102**: O dispositivo deve possuir um botão com sinalização para "UP".
3. **Requisito ID 00103**: O dispositivo deve possuir um botão com sinalização para "DOWN".
4. **Requisito ID 00104**: O dispositivo deve incluir botões físicos 

### Agents

In [8]:
from crewai import Agent, Task, Crew
from langchain.llms import Ollama

# Configurando o Ollama como LLM
llm_ = Ollama(model='llama3.2:1b')
llm1 = Ollama(model="mistral:latest")
llm2 = Ollama(model="qwen:32b")
llm3 = Ollama(model="llama3.3:latest")
llm4 = Ollama(model="deepseek-r1:7b")

# Definindo os agentes
analista_requisitos = Agent(
    role="Analista de Requisitos",
    goal="Analisar clareza e completude dos requisitos de produto",
    backstory="Especialista em engenharia de requisitos com foco em garantir que os requisitos sejam claros, completos e consistentes.",
    llm=llm1,
    verbose=True
)

engenheiro_sistemas = Agent(
    role="Engenheiro de Sistemas",
    goal="Avaliar a viabilidade técnica dos requisitos",
    backstory="Engenheiro experiente em arquitetura de sistemas e desenvolvimento de software, capaz de identificar desafios técnicos e propor soluções.",
    llm=llm2,
    verbose=True
)

revisor_geral = Agent(
    role="Revisor Geral",
    goal="Consolidar as análises e sugerir melhorias nos requisitos",
    backstory="Profissional com ampla experiência em revisão de requisitos e garantia de qualidade, responsável por garantir que os requisitos atendam aos padrões de qualidade.",
    llm=llm3,
    verbose=True,
    allow_delegation=False,
)


# Definindo o limite de interações
max_interactions = 3

# Definindo as tarefas com os requisitos embutidos
tarefa_analise_clareza = Task(
    description=f"Revise os requisitos fornecidos e avalie sua clareza e completude. Identifique quaisquer ambiguidades ou lacunas. Limite as interações com outros agentes a {max_interactions} rodadas.\n\nRequisitos:\n{requisitos_produto}",
    expected_output="Um relatório detalhado sobre a clareza e completude dos requisitos, incluindo sugestões para melhorias.",
    agent=analista_requisitos
)

tarefa_avaliacao_tecnica = Task(
    description=f"Analise os requisitos quanto à viabilidade técnica. Identifique possíveis desafios de implementação e proponha soluções alternativas, se necessário. Limite as interações com outros agentes a {max_interactions} rodadas.\n\nRequisitos:\n{requisitos_produto}",
    expected_output="Um relatório sobre a viabilidade técnica dos requisitos, destacando desafios e soluções propostas.",
    agent=engenheiro_sistemas
)

tarefa_consolidacao = Task(
    description=f"Consolide as análises feitas pelo Analista de Requisitos e pelo Engenheiro de Sistemas. Forneça uma avaliação de cada requisito.\n\nRequisitos:\n{requisitos_produto}",
    expected_output="Um relatório de revisão dos requisitos de produto.",
    agent=revisor_geral
)

# Criando a equipe
equipe_revisao_requisitos = Crew(
    agents=[analista_requisitos, engenheiro_sistemas, revisor_geral],
    tasks=[tarefa_analise_clareza, tarefa_avaliacao_tecnica, tarefa_consolidacao],
    verbose=True
)

# Executando a equipe
resultado = equipe_revisao_requisitos.kickoff()


Working Agent: Analista de Requisitos
Starting Task: Revise os requisitos fornecidos e avalie sua clareza e completude. Identifique quaisquer ambiguidades ou lacunas. Limite as interações com outros agentes a 3 rodadas.

Requisitos:
ID: PRS-00001 ; Requisito: O dispositivo deve ser capaz de operar em cinco modos de operação distintos, que incluam: Modo B, Modo M, Modo Doppler colorido e Doppler power, PW Doppler: 
ID: PRS-00002 ; Requisito: O dispositivo deve permitir a ativação do modo de operação B quando houver uma solicitação através da API: 
ID: PRS-00003 ; Requisito: O dispositivo deve permitir a ativação do modo de operação M quando houver uma solicitação através da solicitação da API: 
ID: PRS-00004 ; Requisito: O dispositivo deve permitir a ativação do modo de operação Color Doppler quando houver uma solicitação através da API: 
ID: PRS-00005 ; Requisito: O dispositivo deve permitir a ativação do modo de operação Power Doppler quando houver uma solicitação através da  API: 
I

In [9]:
# Exibindo o resultado
print("Resultado da Revisão de Requisitos:")
print(resultado)

Resultado da Revisão de Requisitos:
Agent stopped due to iteration limit or time limit.
